# Improving Efficiency 
### Matias Bayas-Erazo

This notebook gives some tips to improve the speed of programs. After introducing vectorization, it deals with some problems that are harder to vectorize. 

#### 1. Vectorizing

Suppose we want to maximize the function $f(x,y)$ over $[-a,a] \times [-a, a]$ where $a = 3$ and $f$ is given by:

$$f(x,y) = \frac{\cos(x^2 + y^2)}{1 + x^2 + y^2}$$

The following code implements a naive maximization routine where we do a grid search over the square to find the maximum value of $f$ over the square:

In [1]:
import numpy as np

def f(x,y):
    return np.cos(x**2 + y**2)/(1 + x**2 + y**2)

grid = np.linspace(-3,3,1000)
m = -np.inf

for x in grid:
    for y in grid:
        z = f(x,y)
        if z > m:
            m = z
print(m)

0.999981964109


The following is a vectorized version of the code above, which runs orders of magnitude faster:

In [2]:
def f(x,y):
    return np.cos(x**2 + y**2)/(1 + x**2 + y**2)

grid = np.linspace(-3,3,1000)
f = np.vectorize(f)
f(grid,grid).max()

0.99998196410857465

#### 2. Numba

When possible, it is always a good idea to vectorize your code. But some problems are harder to vectorize. For example, let's try to generate the trajectory of the following difference equation:

$$ x_{t+1} = 4x_t(1-x_t) $$ 

As a first pass, consider the function gen_diff which generates the trajectory of our difference equation taking as argument the desired length of the path and the initial value:

In [3]:
def gen_diff(x_0, T):
    
    # List to store values:
    x_values = np.empty(T)
    
    # Initial value:
    x_values[0] = x_0
    
    for i in range(T-1):
        x_values[i+1] = 4*x_values[i]*(1-x_values[i])
    
    return x_values

It is not obvious how we can vectorize this code in order to imprtove efficieny. But we can speed this up using Numba, a package that automatically compiles functions into native machine code, helping run the code faster:

In [4]:
from numba import jit
gen_diff_numba = jit(gen_diff) # Creates a compiled version of gen_diff

Let's now time and compare the two functions using identical initial conditions and time series lengths:


In [5]:
%timeit gen_diff(0.1, int(10**5))

10 loops, best of 3: 88.6 ms per loop


In [6]:
%timeit gen_diff_numba(0.1, int(10**5))

The slowest run took 976.55 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 205 µs per loop


The first run is slower because of compilation, if we run the same thing again we get no
warning message and realize that the compiled function increases speed by two orders of magnitude:

In [7]:
%timeit gen_diff_numba(0.1, int(10**5))

1000 loops, best of 3: 218 µs per loop


We don't really need a separate name for the function, we can just compile using numba with the '@' decorator notation:

In [8]:
@jit
def gen_diff(x_0, T):
    
    # List to store values:
    x_values = np.empty(T)
    
    # Initial value:
    x_values[0] = x_0
    
    for i in range(T-1):
        x_values[i+1] = 4*x_values[i]*(1-x_values[i])
    
    return x_values

In [9]:
%timeit gen_diff(0.1, int(10**5))

The slowest run took 434.00 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 218 µs per loop


#### 3. Cython

Another alternative is Cython which allows us to build fast compiled code that can be used from Python. In Cython, we add type definitions directly to our code. Thus, we can think of it as Python with type definitions. We start with the following example. Suppose that for a given $\alpha$ and $n$, we want to compute:

$$ \sum_{i=0}^n \alpha^i $$

Assume we forgot the formula for geometric sums and hence we have to rely on a loop. Consider the following Python code that does this for us:

In [10]:
def geo_sum(alpha, n):
    
    sum = 0
    for i in range(n):
        sum = sum + alpha**i
    return sum

We will now carry out the Cython implementation of the same program written above. We begin by loading the Cython extension in the following cell:

In [11]:
%load_ext Cython

We now execute the following Cython code:

In [12]:
%%cython

def geo_sum_cython(double alpha, int n):
    cdef double sum = 0
    cdef int i
    for i in range(n):
        sum = sum + alpha**i
    return sum

Here, cdef is a keyword indicating variable declaration and is followed by a type. The first line of the cell, %%cython, it's a Jupyter cell magic indicating the beginning of Cython code. Let's now compare the two functions:

In [13]:
%timeit geo_sum(0.99,int(10**6))

10 loops, best of 3: 119 ms per loop


In [14]:
%timeit geo_sum_cython(0.99,int(10**6))

100 loops, best of 3: 16.4 ms per loop


Having introduced basic Cython notation, let's go back to the example on generating a time path for the following difference equation in order to get a grasp of Cython with NumPy arrays. Recall the difference equation was given by:

$$ x_{t+1} = 4x_t ( 1 - x_t) $$

In [15]:
%%cython
import numpy as np 

def gen_cython1(double x_0, int T):
    cdef int i
    x = np.empty(T, float)
    x[0] = x_0
    for i in range(T-1):
        x[i+1] = 4*x[i]*(1-x[i])
    return np.asarray(x)

In [16]:
%timeit gen_cython1(0.1, int(10**5))

10 loops, best of 3: 57.3 ms per loop


Notice that the performance is dissapointing, nothing like the gain in efficiency from using numba. The reason is that we are working with NumPy arrays which incur a lot of overhead. We can do better by using Cython's typed memoryviews, which provide more direct access to arrays in memory. The first step is to create a NumPy array and then declare a memoryview and bind it to the NumPy array. The following code shows how to do this.

In [17]:
%%cython

import numpy as np
from numpy cimport float_t

def gen_cython(double x_0, int T):
    cdef int i
    x_np_array = np.empty(T, float)
    cdef float_t [:] x = x_np_array
    x[0] = x_0
    for i in range(T-1):
        x[i+1] = 4*x[i]*(1-x[i])
    return np.asarray(x)

Going over the code, `cimport` pulls in some compile-time information from NumPy. Then,
```
cdef float_t [:] x = x_np_array
``` 
creates a memoryview on the NumPy array x_np_array. Finally, the `return` statement converts the memoryview back to a NumPy array.


In [18]:
%timeit gen_cython(0.1, int(10**5))

1000 loops, best of 3: 444 µs per loop


Clearly, this helps improve the speed of the program significantly but its still not as fast as `gen_diff_numba`

#### 4. Caching

Suppose you are runnnig a long computation that simulates a model for given parameters and then plots a figure but midway you realize you want to change some feature of your figure and you'll have to do the thing all over again. What caching does is automatically store results at each parameterization. Thus, the second time you run the computation, it will be much faster. We will use the joblib library to do our caching (make sure you have it installed). As an example, we go back to generating a path for the difference equation but now we want to determine what fraction of the points are less than 0.1.



In [20]:
import quantecon as qe
import numpy as np
from scipy.linalg import eigvals
from joblib import Memory

memory = Memory(cachedir='./joblib_cache')

@memory.cache
def gen_diff(x0, n):
    x = np.empty(n+1)
    x[0] = x0
    for t in range(n):
        x[t+1] = 4 * x[t] * (1 - x[t])
    return np.mean(x < 0.1)

qe.util.tic()
n = int(1e7)
print(gen_diff(0.2, n))
qe.util.toc()

________________________________________________________________________________
[Memory] Calling __main__--Users-matiasbayas-Documents-Quant Econ-__ipython-input__.gen_diff...
gen_diff(0.2, 10000000)
_________________________________________________________gen_diff - 8.9s, 0.1min
0.204758079524
TOC: Elapsed: 8.926250219345093 seconds.


8.926250219345093

We use joblib to cache the result of calling the function `gen_diff` for a given set of parameters. With the argument `cachedir='./joblib_cache'` any call to this function results in the input and output values being stored in a subdirectory of the present working directory. If we call the function a second time, we'll see how the timing improves significantly:

In [21]:
qe.util.tic()
n = int(1e7)
print(gen_diff(0.2, n))
qe.util.toc()

0.204758079524
TOC: Elapsed: 0.0017337799072265625 seconds.


0.0017337799072265625